In [8]:
import splitfolders
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Split dataset into train (70%), val (20%), test (10%) with stratified sampling
original_dataset_dir = "./dataset"
output_dir = "./datasplit"

splitfolders.ratio(
    original_dataset_dir,
    output=output_dir,
    seed=42,
    ratio=(0.7, 0.2, 0.1),
    move=False,
)

# Define paths
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

# Image dimensions and parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-4
train_datagen = ImageDataGenerator(
    preprocessing_function=lambda x: x / 255.0,  # Manually rescale inputs
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.8, 1.2),
    horizontal_flip=True,
    fill_mode="nearest",
)


val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # This outputs one-hot labels
)

# Validation generator
val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Match the same structure
)

# Test generator
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Consistent class_mode
    shuffle=False
)


Found 483 images belonging to 4 classes.
Found 137 images belonging to 4 classes.
Found 74 images belonging to 4 classes.


In [9]:

# Calculate class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_generator.classes),
    y=train_generator.classes,
)
class_weights = dict(enumerate(class_weights))
print(f"Class weights: {class_weights}")

# Load MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze base model
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



Class weights: {0: 1.0320512820512822, 1: 0.966, 2: 1.040948275862069, 3: 0.966}


In [12]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model with frozen layers
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.2930 - loss: 1.7960 - val_accuracy: 0.2701 - val_loss: 1.4294 - learning_rate: 1.0000e-04
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - accuracy: 0.3389 - loss: 1.4899 - val_accuracy: 0.3358 - val_loss: 1.2793 - learning_rate: 1.0000e-04
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.4103 - loss: 1.3187 - val_accuracy: 0.4015 - val_loss: 1.2334 - learning_rate: 1.0000e-04
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.4095 - loss: 1.2827 - val_accuracy: 0.3942 - val_loss: 1.1861 - learning_rate: 1.0000e-04
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.4304 - loss: 1.3067 - val_accuracy: 0.4161 - val_loss: 1.1486 - learning_rate: 1.0000e-04
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.4951 - loss: 1.1083 - val_accuracy: 0.4307 - val_loss: 1.1474 - learning_rate: 1.0000e-04
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.5112 - loss:

In [13]:

# Fine-tune the base model
base_model.trainable = True

# Freeze earlier layers for gradual unfreezing
for layer in base_model.layers[:60]:
    layer.trainable = False

# Re-compile for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:


# Fine-tune the model
history_fine_tune = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - accuracy: 0.4272 - loss: 1.2831 - val_accuracy: 0.5182 - val_loss: 1.0482 - learning_rate: 1.0000e-05
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.4431 - loss: 1.2319 - val_accuracy: 0.5036 - val_loss: 1.0889 - learning_rate: 1.0000e-05
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.5355 - loss: 1.0966 - val_accuracy: 0.4891 - val_loss: 1.1328 - learning_rate: 1.0000e-05
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.5341 - loss: 1.0343 - val_accuracy: 0.4891 - val_loss: 1.1810 - learning_rate: 1.0000e-05
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.5659 - loss: 0.9958 - val_accuracy: 0.4745 - val_loss: 1.1833 - learning_rate: 2.0000e-06


In [16]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 417ms/step - accuracy: 0.5650 - loss: 1.0885
Test Accuracy: 0.57


In [17]:

# Confusion Matrix and Classification Report
from sklearn.metrics import classification_report, confusion_matrix

# Predictions
y_pred = np.argmax(model.predict(test_generator), axis=-1)
print("Confusion Matrix:")
print(confusion_matrix(test_generator.classes, y_pred))
print("\nClassification Report:")
print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))


3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step  
Confusion Matrix:
[[ 9  4  4  1]
 [ 4  9  4  2]
 [ 0  0 18  0]
 [ 5  4  4  6]]

Classification Report:
              precision    recall  f1-score   support

       Angry       0.50      0.50      0.50        18
        Fear       0.53      0.47      0.50        19
       Happy       0.60      1.00      0.75        18
         Sad       0.67      0.32      0.43        19

    accuracy                           0.57        74
   macro avg       0.57      0.57      0.54        74
weighted avg       0.57      0.57      0.54        74

